In [2]:
import pandas as pd
df = pd.read_csv(r"Preprocessed_Data\unscaled_filtered.csv", index_col=0)

In [3]:
import pandas as pd
from opendata import OpenData
from concurrent.futures import ThreadPoolExecutor, as_completed

# 1) Your list of athlete IDs (already cleaned, no braces):
athlete_ids = list(df.index)

# 2) Helper: fetch all DataFrames for one athlete
def fetch_one_athlete(athlete):
    athlete_id = athlete.metadata['ATHLETE']['id'].strip("{}")
    dfs = []
    for activity in athlete.activities():
        df = activity.data
        df = df.assign(
            athlete_id=athlete_id,
            date=pd.to_datetime(activity.metadata.get('date'))
        )
        dfs.append(df)
    return dfs

# 3) Initialize client and build athlete objects
od = OpenData()
athletes = []
for aid in athlete_ids:
    try:
        athletes.append(od.get_local_athlete(aid))
    except Exception as e:
        print(f"⚠️ Failed to fetch {aid}: {e}")

# 4) Pull all activities in parallel
all_dfs = []
with ThreadPoolExecutor(max_workers=8) as exe:
    futures = {exe.submit(fetch_one_athlete, ath): ath for ath in athletes}
    for fut in as_completed(futures):
        try:
            all_dfs.extend(fut.result())
        except Exception as e:
            bad = futures[fut].metadata['ATHLETE']['id']
            print(f"Error fetching activities for {bad}: {e}")

# 5) Concatenate into one big DataFrame
all_activities_df = pd.concat(all_dfs, ignore_index=True)
print(all_activities_df[['athlete_id','date']].shape)


Error fetching activities for {aa032581-fc55-457d-a4cf-c3e80cdd1749}: 'NoneType' object has no attribute 'get'
Error fetching activities for {455b797a-4e0b-46c1-bf84-bb28c8ddbeff}: 'NoneType' object has no attribute 'get'
Error fetching activities for {525d1d99-83d4-48bd-8603-339c065a59c6}: 'NoneType' object has no attribute 'get'
Error fetching activities for {b9a1d1e3-908f-4b56-803b-daae66a24ed6}: 'NoneType' object has no attribute 'get'
Error fetching activities for {c5302ad9-dd0d-4307-a1dd-25489c3da0e7}: 'NoneType' object has no attribute 'get'
Error fetching activities for {3991206e-aeaa-4de8-9626-370a0a9cbcd9}: 'NoneType' object has no attribute 'get'
Error fetching activities for {61c5a889-9114-4a85-8285-c8f7d1a80d81}: 'NoneType' object has no attribute 'get'
Error fetching activities for {f8d152df-f14b-4247-b3d6-bb537b9630d8}: 'NoneType' object has no attribute 'get'
Error fetching activities for {162de289-037a-41f9-9ad0-8554e32fc615}: 'NoneType' object has no attribute 'get'
E

MemoryError: 

In [ ]:
+